# **Bulldozer sales price estimation using Machine Learning**
This notebook explores some of the basic datascience and machine learning concepts by trying to solve the problem of Bulldozers sales price prediction.  
Since the desired outcome here is an estimate of future sales price(predicting a number) this is a **Regression** problem.  

**Problem statement:**  
> Can the future sales price of a bulldozer be predicted the machine learning techniques and models?  

**Data to solve the problem:**  
The data is taken from the *Blue Book for Bulldozers Kaggle Competition*.[(link here)](https://www.kaggle.com/c/bluebook-for-bulldozers/data)  
The data for this competition is split into three parts:

1. Train.csv is the training set, which contains data through the end of 2011.
2. Valid.csv is the validation set, which contains data from January 1, 2012 - April 30, 2012.
3. Test.csv is the test set. It contains data from May 1, 2012 - November 2012.

The key fields are in train.csv are:

- SalesID: the uniue identifier of the sale
- MachineID: the unique identifier of a machine.  A machine can be sold multiple times
- saleprice: what the machine sold for at auction (only provided in train.csv)
- saledate: the date of the sale  

Other than this there is a [data dictionary](https://drive.google.com/file/d/1qhPAQSFWw3rgpiMy88tMJInP0VGmoSj5/view?usp=sharing) provided with this dataset to understand more about the features of the data.  

---

## 0. Importing all necessary tools

In [1]:
# Data analysis and plotting
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Regression models from Scikit-learn library
from sklearn.ensemble import RandomForestRegressor

# Model evaluation and experimentation
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, r2_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, cross_val_score

# saving the model
import joblib

## Data loadind and EDA
**Note:**  
> Data files are downloaded from [kaggle competition page](https://www.kaggle.com/c/bluebook-for-bulldozers/data) and uploaded to google drive in order to import and use it here in this notebook.  
Anyone can view the data folder used here, from this [link.](https://drive.google.com/drive/folders/1GzNp18zvT9JBbEsyjYSQnk_6mUecO6Aq?usp=sharing)  

Loading the dataset from the data folder and saving it as Pandas DataFrame. Then perform some exploratory data analysis(EDA) on the saved DataFrame.